In [2]:
# importing all necessary libraries
import mysql
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pandas as pd
import sqlalchemy #version 1.4.46


In [3]:
# Function to extract table to a pandas DataFrame
def extract_table_to_pandas(tablename, db_engine_1):
    query = "SELECT * FROM {}".format(tablename)
    return pd.read_sql(query, db_engine_1)

db_engine_1 = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                                   format('subnet_user', 'subnet_user123', 
                                                          'localhost', 'elearning'))

### Funtion to extract useful information from database

In [4]:
def extract_rating_data(db_engine_1):
    return pd.read_sql("SELECT * FROM rating", db_engine_1)

def extract_course_data(db_engine_1):
    return pd.read_sql("SELECT * FROM courses", db_engine_1)

In [5]:
ratings = extract_rating_data(db_engine_1)

course_data = extract_course_data(db_engine_1)

In [6]:
ratings.head()

,user_id,course_id,rating
0,1,6,4
1,1,36,5
2,1,37,5
3,1,45,5
4,1,50,5


In [7]:
course_data.head()

,course_id,title,description,programming_language
0,1,Machine Learning with Apache Spark,"Spark is a powerful, general purpose tool for ...",python
1,2,Financial Analytics in Spreadsheets,Monitoring the evolution of traded assets is k...,spreadsheets
2,3,Intermediate R,The intermediate R course is the logical next ...,r
3,4,Data Visualization with ggplot2 (Part 2),This ggplot2 tutorial builds on your knowledge...,r
4,5,Fraud Detection in R,The Association of Certified Fraud Examiners e...,r


In [8]:
# pl = course_data['programming_language']
# pl = pd.DataFrame(pl)
# pl

In [9]:
ratings.user_id.nunique()

38174

In [10]:
course_data.course_id.nunique()

100

In [11]:
course_data.isnull().sum()

course_id               0
title                   0
description             0
programming_language    3
dtype: int64

### filling null values in programming language wirh 'r'

In [12]:
# The transformation should fill in the missing values
def transform_fill_programming_language(course_data):
    imputed = course_data.fillna({"programming_language": "r"})
    return imputed

transformed = transform_fill_programming_language(course_data)

# Print out the number of missing values per column of transformed
print(transformed.isnull().sum())

course_id               0
title                   0
description             0
programming_language    0
dtype: int64


### Showing total programming languages in percentage

In [13]:
course_data['programming_language'].unique()

array(['python', 'spreadsheets', 'r', 'sql', 'shell', None], dtype=object)

In [14]:
(course_data['programming_language'].value_counts()/course_data['programming_language'].count())*100

r               51.546392
python          37.113402
sql              6.185567
shell            3.092784
spreadsheets     2.061856
Name: programming_language, dtype: float64

## finding most rated course in databse
## transformation function

In [15]:
# transformation function
def transform_avg_rating(rating_data):
    # Group by course_id and extract average rating per course
    avg_rating = rating_data.groupby('course_id').rating.mean()
    # Return sorted average ratings per course
    sort_rating = avg_rating.sort_values(ascending=False).reset_index()
    return sort_rating

In [16]:
# Use transform_avg_rating on the extracted data and print results
avg_course_ratings = transform_avg_rating(ratings)
print(avg_course_ratings) 

    course_id    rating
0          46  4.800000
1          23  4.800000
2          96  4.692765
3          56  4.661765
4          24  4.653061
..        ...       ...
94         54  4.238095
95         92  4.222222
96         29  4.208333
97         17  4.147059
98         42  4.107570

[99 rows x 2 columns]


### Creating database of users and the course id's in which they are not enrolled 

In [20]:
def get_courses_to_recommend(ratings, course_data):
    user_list = ratings.user_id.unique()
    #38174
    
    languages_set = set(course_data.programming_language.unique())    
    course_set = set(course_data.course_id.unique())
    
    non_enrolled_users = []

    for user in user_list:
        rated_course_set = set(ratings[ratings.user_id == user].course_id.unique())
        non_enrolled_courses = course_set - rated_course_set
        language_enrolled_set = set(course_data[course_data.course_id.isin(rated_course_set)].programming_language.unique())
        non_enrolled_languages = languages_set - language_enrolled_set
        non_enrolled_users = non_enrolled_users + [(user, cid) for cid in non_enrolled_courses]
        break
    non_enrolled_courses_df = pd.DataFrame(non_enrolled_users, columns = ['user_id', 'course_id'])
    return non_enrolled_courses_df

courses_to_recommend = get_courses_to_recommend(ratings, course_data)
courses_to_recommend.to_csv('courses_to_recommend.csv', index = False)
courses_to_recommend.sample(n=50).to_csv('courses_to_recommend_small.csv', index = False)

In [22]:
len(courses_to_recommend)
# 3854714

94

In [23]:
courses_to_recommend.head(10)

,user_id,course_id
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
5,1,7
6,1,8
7,1,9
8,1,10
9,1,11


In [26]:
courses_to_recommend[courses_to_recommend['user_id']==1]

,user_id,course_id
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
89,1,96
90,1,97
91,1,98
92,1,99


In [27]:
# Complete the transformation function
def transform_recommendations(avg_course_ratings, courses_to_recommend):
    # Merge both DataFrames
    merged = courses_to_recommend.merge(avg_course_ratings) 
    
    # Sort values by rating and group by user_id
    grouped = merged.sort_values("rating", ascending=False).groupby("user_id")
    
    # Produce the top 3 values and sort by user_id
    recommendations = grouped.head(10).sort_values("user_id").reset_index()
    
    final_recommendations = recommendations[["user_id", "course_id","rating"]]
    
    # Return final recommendations
    return final_recommendations


# Use the function with the predefined DataFrame objects
recommendations = transform_recommendations(avg_course_ratings, courses_to_recommend)

In [28]:
recommendations

,user_id,course_id,rating
0,1,23,4.800000
1,1,46,4.800000
2,1,96,4.692765
3,1,56,4.661765
4,1,24,4.653061
5,1,26,4.646259
6,1,61,4.629213
7,1,85,4.627119
8,1,87,4.626374
9,1,31,4.610209


In [ ]:
# def a(x):
#     return [x,course_data[course_data.course_id == x].programming_language]
# recomend = list(map(a,recommendations.course_id))

In [83]:
# recommendations = pd.merge(recommendations,course_data, on='course_id', how='inner')
# recommendations

,user_id,course_id,rating,title,description,programming_language
0,1,24,4.653061,Nonlinear Modeling in R with GAMs,Generalized Additive Models are a powerful too...,r
1,2,24,4.653061,Nonlinear Modeling in R with GAMs,Generalized Additive Models are a powerful too...,r
2,3,24,4.653061,Nonlinear Modeling in R with GAMs,Generalized Additive Models are a powerful too...,r
3,4,24,4.653061,Nonlinear Modeling in R with GAMs,Generalized Additive Models are a powerful too...,r
4,5,24,4.653061,Nonlinear Modeling in R with GAMs,Generalized Additive Models are a powerful too...,r
...,...,...,...,...,...,...
381735,16456,28,4.578947,Python for Spreadsheet Users,Are you looking for a better solution than the...,python
381736,12087,35,4.588235,Interactive Data Visualization with rbokeh,Data visualization is an integral part of the ...,r
381737,16456,35,4.588235,Interactive Data Visualization with rbokeh,Data visualization is an integral part of the ...,r
381738,25147,35,4.588235,Interactive Data Visualization with rbokeh,Data visualization is an integral part of the ...,r


### Load recommendations in database

In [29]:
def load_to_dwh(recommendations, db_engine_1):
    recommendations.to_sql("recommendations", db_engine_1, if_exists="replace")

In [30]:
load_to_dwh(recommendations, db_engine_1)

### Function to recommend courses to user based on user's choice of programming language

In [38]:
def recommendations_for_user(user_id, programming_language, threshold ):
    # Join with the courses table
    query = """
    SELECT title, rating FROM recommendations
    INNER JOIN courses ON courses.course_id = recommendations.course_id
    WHERE (user_id=%(user_id)s AND programming_language=%(programming_language)s AND rating>%(threshold)s) 
    ORDER BY rating DESC
    """
    # Add the threshold parameter
    predictions_df = pd.read_sql(query, db_engine_1, params = {"user_id": user_id, 
                                                           "threshold": threshold,
                                                            "programming_language":programming_language})
    return predictions_df.title.values

# Try the function you created
print(recommendations_for_user(1,'r',4.1))


['GARCH Models in R' 'Designing and Analyzing Clinical Trials in R'
 'Nonlinear Modeling in R with GAMs'
 'Manipulating Time Series Data in R: Case Studies'
 'Bond Valuation and Analysis in R'
 'Introduction to Spark in R using sparklyr'
 'Introduction to R for Finance']


In [97]:
print(recommendations_for_user(9999,'r',4.7))

['GARCH Models in R' 'Designing and Analyzing Clinical Trials in R']


In [71]:
print(recommendations_for_user(1,'python',4.3))

['Extreme Gradient Boosting with XGBoost' 'Introduction to Seaborn']


In [104]:
print(recommendations_for_user(9999,'sql',4.3))

['Intro to SQL for Data Science']


## ETL: Combining all operations

In [ ]:
def etl(db_engine_1):
    # Extract the data
    courses = extract_course_data(db_engine_1)
    rating = extract_rating_data(db_engine_1)
    
    # Clean up courses data
    courses = transform_fill_programming_language(courses)
    
    # Get the average course ratings
    avg_course_rating = transform_avg_rating(rating)
    
    # Get eligible user and course id pairs
    courses_to_recommend = pd.read_csv('courses_to_recommend.csv')
    
    # Calculate the recommendations
    recommendations = transform_recommendations(
        avg_course_rating,
        courses_to_recommend
    )
    
    # Load the recommendations into the database
    load_to_dwh(recommendations, db_engine_1)


In [ ]:
etl(db_engine)

### Calling main function

In [119]:
a = int(input('enter user id:'))
b = str(input('enter language:'))
c = float(input('enter minimum rating of course:'))
print(recommendations_for_user(a,b,c))

enter user id:27894
enter language:python
enter minimum rating of course:3.9
['Extreme Gradient Boosting with XGBoost' 'Introduction to Seaborn']


In [114]:
a = int(input('enter user id:'))
b = str(input('enter language:'))
c = float(input('enter minimum rating of course:'))
print(recommendations_for_user(a,b,c))

enter user id:1
enter language:sql
enter minimum rating of course:4.0
['Intro to SQL for Data Science']
